# Install Necessary Libraries

In [ ]:
pip install openpyxl anonympy xlrd>=1.0.0

In [ ]:
pip install cape-privacy --no-deps

# Get File Meta Data

Enter the path of the data feeds

In [1]:
root = ''

Save a list of the feeds and their attributes to a CSV file and display the results.

In [7]:
import pandas as pd
import os

pd.set_option('display.max_rows', None)

full_path = []

for path, subdirs, files in os.walk(root + 'Input_Files'):
    for name in files:
        if name != '.DS_Store' and '.asc' not in name:
            full_path.append(os.path.join(path, name))
df = pd.DataFrame(full_path, columns=['full_path'])
df = df['full_path'].str.split('/',expand=True)
# df = df.drop([1,2,3,4], axis=1)
df = df.drop([0], axis=1)
df = df.rename(columns={1:'object',2:'customer',3:'file'})
df['is_gpg'] = df['file'].str.contains('.gpg')
df['temp_file'] = df['file'].replace('.gpg','', regex=True)
df['dots'] = df['temp_file'].str.split('.').str.len()
df['file_type'] = df.apply(lambda x: x['temp_file'].split('.')[x['dots']-1].lower(), axis=1)
df = df.drop(['dots', 'temp_file'], axis=1)

df.to_csv('files.csv')

df_current = pd.read_excel('data_files.xlsx', sheet_name='files', header=0)

df_not_listed = df.merge(df_current.drop_duplicates(), on=['object','customer','file'], 
                   how='left', indicator=True)

df_not_listed[df_not_listed['_merge']=='left_only'][['object','customer','file','is_gpg','file_type_x']]

,object,customer,file,is_gpg,file_type_x
7,Claims-Medical,ND-BCBS (Medicaid),BCBSND_PRVGROUP_20230201.txt,False,txt
10,Claims-Medical,ND-BCBS (Medicaid),BCBSND_MED_CLM_20230201.txt,False,txt
11,Claims-Medical,ND-BCBS (Medicaid),BCBSND_PRV_20230201.txt,False,txt
24,ADT,HealthieNevada,Roster Specifications Flat File Only v1.0 FINA...,False,docx
28,ADT,Bamboo,Bamboo Health Outbound HL7 Integrations Proces...,False,docx
39,ADT,Experian,Updated_MemberMatch - Roster Sample File Layou...,False,txt
41,ADT,HealthCurrent,Health Current Patient Panel Request CSV File....,False,xlsx
76,Quality,Humana,Humana_Arkos_Hedis_202308.txt,False,txt
89,Carepointe,All,revenue_pmpm.csv,False,csv
95,Carepointe,All,quality-gap-report-by-patient+measure-cut-poin...,False,csv


In [13]:
import pandas as pd

# gsheet_name = "files"
# files = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={gsheet_name}"
# df_files = pd.read_csv(files)
# df_files = df_files.rename(columns={'Unnamed: 8':'sheet_id', 'Unnamed: 9':'header_row'})

start = 172
end = 176

df_files = pd.read_excel('data_files.xlsx', sheet_name='files', header=0)[start:end+1]
df_files

,0,object,customer,file,encr,file_type,delimiter,sheet_name,sheet_id,header_row,parent_folder,child_folder,skip,notes
172,172,Quality,Humana,Humana_Arkos_Hedis_202308.txt,False,txt,tab,NaN,NaN,NaN,Humana,Quality,no,NaN
173,173,Census,Humana,Arkos_Census_20230918.txt,False,txt,pipe,NaN,NaN,NaN,Humana,Census,no,NaN
174,174,Census,Humana,Arkos_Census_20230916.txt,False,txt,pipe,NaN,NaN,NaN,Humana,Census,no,NaN
175,175,Census,Humana,Arkos_Census_20230917.txt,False,txt,pipe,NaN,NaN,NaN,Humana,Census,no,NaN
176,176,Risk,Humana,Humana_Arkos_HCC_202309.txt,False,txt,tab,NaN,NaN,NaN,Humana,Risk,no,NaN


Extract all the fields from each feed and put them into a CSV file

In [51]:
# get list of all fields in all files

import pandas as pd
import numpy as np

data_dict = pd.DataFrame()

delimiters = {
    "comma":",",
    "pipe":"|",
    "tab":"\t"
}

for r in range(0, len(df_files.index)):
    object = df_files.iloc[r]['object']
    customer = df_files.iloc[r]['customer']
    parent_folder = df_files.iloc[r]['parent_folder']
    child_folder = df_files.iloc[r]['child_folder']
    file = df_files.iloc[r]['file']
    encr = df_files.iloc[r]['encr']
    file_type = df_files.iloc[r]['file_type']
    delimiter_name = df_files.iloc[r]['delimiter']
    delimiter = delimiters[delimiter_name]
    sheet_id = np.nan_to_num(df_files.iloc[r]['sheet_id']).astype(int)
    header_row = np.nan_to_num(df_files.iloc[r]['header_row']).astype(int)
    skip = df_files.iloc[r]['skip']
    notes = df_files.iloc[r]['notes']

    file = file.replace('.gpg', '')
    f = 'Input_Files/'+object+'/'+customer+'/'+file

    dd = pd.DataFrame(columns=['object','customer','file','file_type','sheet_id', 'field','type','sample'])

    if file_type.lower() == 'csv':
        df = pd.read_csv(f, nrows=10)
        fields = df.dtypes.to_frame('dtypes').reset_index()
        dd['field'] = fields['index']
        dd['type'] = fields['dtypes']
    if file_type.lower() == 'txt' and len(file.split('.')) < 3:
        df = pd.read_table(f, delimiter=delimiter, nrows=10, encoding='latin_1')
        fields = df.dtypes.to_frame('dtypes').reset_index()
        dd['field'] = fields['index']
        dd['type'] = fields['dtypes']
    if file_type.lower() == 'xlsx':
        df = pd.read_excel(f, sheet_name=sheet_id, nrows=20, header=header_row)
        fields = df.dtypes.to_frame('dtypes').reset_index()
        dd['field'] = fields['index']
        dd['type'] = fields['dtypes']
    if file_type.lower() == 'xls':
        df = pd.read_excel(f, sheet_name=sheet_id, nrows=10, engine='xlrd')
        fields = df.dtypes.to_frame('dtypes').reset_index()
        dd['field'] = fields['index']
        dd['type'] = fields['dtypes']

    dd['object'] = object
    dd['customer'] = customer
    dd['file'] = file
    dd['file_type'] = file_type
    dd['sheet_id'] = sheet_id
    dd['sample'] = df.loc[0, :].values.tolist()

    data_dict = pd.concat([data_dict, dd])

data_dict

# data_dict.to_csv('data_files.csv')


,object,customer,file,file_type,sheet_id,field,type,sample
0,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,VIEW_DESC,object,Quest BY2026 Current Year
1,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MEASUREMENT_YEAR,int64,2023
2,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MBR_PERS_GEN_KEY,int64,1011004739522
3,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,SDR_PERSON_ID,int64,1102798939
4,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MEDICARE_ID,object,9AP4J40YQ72
5,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,IDCARD_MBR_ID,object,H7295131300
6,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MBR_FIRST_NAME,object,DARRYL
7,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MBR_LAST_NAME,object,SMITH
8,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MBR_BIRTH_DATE,object,1961-12-30
9,Quality,Humana,Humana_Arkos_Hedis_202308.txt,txt,0,MEASURE_CAT,object,FLU_2023


# Mask Data

Before masking the data, the csv files with feed names and feed columns must be loaded into a Google sheet so that masking rules can be defined.  These rules are based off of the masking functions available in the `anonympy` library.

This library will keep all the structure and rows of the feed files, but mask columns specified with masking rules.

Functions to create directories if they don't exist and to mask data

In [52]:
import gpg_lite as gpg

def create_directory(path):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
        
def mask_data(df, date_fields, token_fields, fake_fields):
    # ensure data types are correct for masked fields
    df[date_fields] = df[date_fields].astype('datetime64[ns]')
    df[token_fields] = df[token_fields].astype(str)

    # mask data in current file per rules
    anonym = dfAnonymizer(df)

    anonym.categorical_fake(fake_fields)
    anonym.datetime_noise(date_fields)
    anonym.categorical_tokenization(token_fields, 12, key='bsNhFSaYSU')

    dfm = anonym.to_df()
    dfm = dfm[df.notna()]
    
    return dfm

Loop through each row feed list and apply respective masking rules in the list of columns

In [55]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
import json
import pandas as pd

from anonympy.pandas import dfAnonymizer 

# sheet_name = "mapping_rules"
# map_file = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
# df_map_file = pd.read_csv(map_file)
df_map_file = pd.read_excel('data_files.xlsx', sheet_name="mapping_rules")

delimiters = {
    "comma":",",
    "pipe":"|",
    "tab":"\t"
}

for r in range(0, len(df_files.index)):
    object = df_files.iloc[r]['object']
    customer = df_files.iloc[r]['customer']
    parent_folder = df_files.iloc[r]['parent_folder']
    child_folder = df_files.iloc[r]['child_folder']
    file = df_files.iloc[r]['file']
    encr = df_files.iloc[r]['encr']
    file_type = df_files.iloc[r]['file_type']
    delimiter_name = df_files.iloc[r]['delimiter']
    delimiter = delimiters[delimiter_name]
    sheet_id = np.nan_to_num(df_files.iloc[r]['sheet_id']).astype(int)
    header_row = np.nan_to_num(df_files.iloc[r]['header_row']).astype(int)
    skip = df_files.iloc[r]['skip']
    notes = df_files.iloc[r]['notes']
    
    file = file.replace('.gpg', '')
    f = 'Input_Files/'+object+'/'+customer+'/'+file
    output_path = root + 'Output_Files/'+parent_folder+'/'+child_folder+'/'+file+''
    
    if skip == 'yes':
        print('skipping ',file, 'due to', notes)
    else:
        print('start',file)

        # create sub folders if they don't exist and save results back to a file
        path = root + 'Output_Files'
        create_directory(path)
        path = root + 'Output_Files/'+parent_folder
        create_directory(path)
        path = root + 'Output_Files/'+parent_folder+'/'+child_folder
        create_directory(path)

        # get masking rules for current file
        dfm = df_map_file[df_map_file['customer'].eq(customer) & df_map_file['object'].eq(object) & df_map_file['file'].eq(file)]

        date_fields = dfm[dfm['mask method'].eq('datetime_noise')]['field']

        token_fields = dfm[dfm['mask method'].eq('categorical_tokenization')]['field']

        fake_fields = dfm[dfm['mask method'].eq('categorical_fake')][['field', 'mask type']]\
            .set_index('field')\
            .to_json(orient='columns')

        fake_fields = json.loads(fake_fields)['mask type']

        #read data from current file into dataframe

        if file_type.lower() == 'csv':
            df = pd.read_csv(f)
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_csv(root + 'Output_Files/'+parent_folder+'/'+child_folder+'/'+file+'', index=False)

        #### allow for tab delimited and fixed with files ####
        if file_type.lower() == 'txt' and len(file.split('.')) < 3:
            df = pd.read_table(f, delimiter=delimiter, encoding='latin_1')
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_csv(root + 'Output_Files/'+parent_folder+'/'+child_folder+'/'+file+'', sep=delimiter, index=False)

        #### mask excel files ####
        if file_type.lower() == 'xlsx':
            df = pd.read_excel(f, sheet_name=sheet_id, header=header_row)
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_excel(root + 'Output_Files/'+parent_folder+'/'+child_folder+'/'+file+'', index=False)
        if file_type.lower() == 'xls':
            df = pd.read_excel(f, sheet_name=sheet_id, header=header_row, engine='xlrd')
            dfm = mask_data(df, date_fields, token_fields, fake_fields)
            dfm.to_excel(root + 'Output_Files/'+parent_folder+'/'+child_folder+'/'+file+'x', index=False)

        print('end',file)

        

/Users/anthonychamberas/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


start Humana_Arkos_Hedis_202308.txt


/var/folders/xr/smm2095x7txfhj4s2x6wzqyw0000gn/T/ipykernel_87526/2152423634.py:75: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(f, delimiter=delimiter, encoding='latin_1')


end Humana_Arkos_Hedis_202308.txt
start Arkos_Census_20230918.txt
end Arkos_Census_20230918.txt
start Arkos_Census_20230916.txt
end Arkos_Census_20230916.txt
start Arkos_Census_20230917.txt
end Arkos_Census_20230917.txt
start Humana_Arkos_HCC_202309.txt
end Humana_Arkos_HCC_202309.txt


# Tests and Expirements (Ignore)

In [7]:
# https://levelup.gitconnected.com/python-data-anonymization-masking-guide-de0b0aa0ca82

from anonympy.pandas.utils_pandas import available_methods
from anonympy.pandas.utils_pandas import fake_methods

print(available_methods())
print(fake_methods())

`numeric`:
        * Perturbation - "numeric_noise"
        * Binning - "numeric_binning"
        * PCA Masking - "numeric_masking"
        * Rounding - "numeric_rounding"

`categorical`:
        * Synthetic Data - "categorical_fake"
        * Synthetic Data Auto - "categorical_fake_auto"
        * Resampling from same Distribution - "categorical_resampling"
        * Tokenazation - "categorical_tokenization"
        * Email Masking - "categorical_email_masking"

`datetime`:
        * Synthetic Date - "datetime_fake"
        * Perturbation - "datetime_noise"

`general`:
        * Drop Column - "column_suppression"
        
None
A | aba, address, administrative_unit, am_pm, android_platform_token, ascii_company_email, ascii_email, ascii_free_email, ascii_safe_email
B | bank_country, bban, boolean, bothify, bs, building_number
C | cache_pattern, catch_phrase, century, chrome, city, city_prefix, city_suffix, color, color_name, company, company_email, company_suffix, coordinate, country, c

In [6]:
import pandas as pd
from anonympy.pandas import dfAnonymizer 

df1 = pd.DataFrame([{"id":"ABCDEF","sex":"M"},{"id":"ABCDEF","sex":"M"},{"id":"UVWXYZ","sex":"F"}])
df2 = pd.DataFrame([{"id":"ABCDEF"},{"id":"ABCDEF"},{"id":"GHIJKL"}])

anonym1 = dfAnonymizer(df1)
anonym1.categorical_tokenization('id', 12, key='testkey')
#anonym1.categorical_fake({'sex':'random_letter'})
anonym1.categorical_resampling('sex')

df1['masked_id'] = anonym1.to_df()['id']
df1['masked_sex'] = anonym1.to_df()['sex']
print(df1)
print('\n')

anonym2 = dfAnonymizer(df2)
anonym2.categorical_tokenization('id', 12, key='testkey')

df2['masked_id'] = anonym2.to_df()['id']
print(df2)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


       id sex     masked_id masked_sex
0  ABCDEF   M  50b95426ef40          M
1  ABCDEF   M  50b95426ef40          M
2  UVWXYZ   F  51dfcd54c153          M


       id     masked_id
0  ABCDEF  50b95426ef40
1  ABCDEF  50b95426ef40
2  GHIJKL  5dbeec07962e


In [ ]:
encrypted_file = '/Users/anthonychamberas/Desktop/Input_Files/Claims-Medical/ND-BCBS (Medicaid)/BCBSND_MED_CLM_20230201.txt.gpg'
decrypt(encrypted_file)

gpg --decrypt 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_MED_CLM_20230201.txt.gpg' > 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_MED_CLM_20230201.txt.gpg'
gpg --decrypt 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRV_20230201.txt.gpg' > 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRV_20230201.txt'
gpg --decrypt 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRVGROUP_20230201.txt.gpg' > 'Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRVGROUP_20230201.txt'
gpg --decrypt 'Claims-RX/ND BCBS/BCBSND_RX_CLM_20230301.txt.gpg' > 'Claims-RX/ND BCBS/BCBSND_RX_CLM_20230301.txt'
gpg --decrypt 'Claims-RX/ND NextBlue/NBND_2023_MA_Pharmacy_V7.12_20230220103007.txt.gpg' > 'Claims-RX/ND NextBlue/NBND_2023_MA_Pharmacy_V7.12_20230220103007.txt'


In [ ]:
import gpg_lite as gpg

gpg_store = gpg.GPGStore()

encrypted_file = '/Users/anthonychamberas/Downloads/Input_Files/Claims-Medical/ND-BCBS (Medicaid)/BCBSND_PRVGROUP_20230201.txt.gpg'
decrypted_file = encrypted_file[:-4]
with open(encrypted_file, "rb") as f, open(decrypted_file, "w") as f_out:
    gpg_store.decrypt(
      source=f,
      output=f_out,
      passphrase="Chuck Norris does not need one - the password needs him")

with open(decrypted_file, "r") as f:
    print("Decrypted message:", "".join(f.readlines()))

In [ ]:
def decrypt(encrypted_file):
    gpg_store = gpg.GPGStore()
    
    decrypted_file = encrypted_file[:-4]
    with open(encrypted_file, "rb") as f, open(decrypted_file, "w") as f_out:
        gpg_store.decrypt(
          source=f,
          output=f_out,
          passphrase="Chuck Norris does not need one - the password needs him")
        
    return ('decrypted', encrypted_file)
